In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from scipy import sparse
import warnings
import time
import sys
import os
import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss

In [ ]:
train_df = pd.read_csv('dataset/train_dataset.csv')
test_df = pd.read_csv('dataset/test_dataset.csv')

In [ ]:
def score(pre, truth):
    return 1/(MAE(pre, truth) + 1)

def MAE(pre, truth):
    return abs((pre.astype(int) - truth)).mean()

In [ ]:
def lgb_model(param, X_train, y_train, X_test, name):
    # 五折交叉验证
    folds = KFold(n_splits=5, shuffle=True, random_state=201)
    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
        print("fold n°{}".format(fold_+1))
        trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
        val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

        num_round = 10000
        clf = lgb.train(param, 
                        trn_data, 
                        num_round, 
                        valid_sets = [trn_data, val_data], 
                        verbose_eval = 200, 
                        early_stopping_rounds = 100)
        oof[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)

        predictions += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
        
    sub_df = pd.read_csv('dataset/submit_example.csv')
    sub_df[' score'] = predictions.astype(int)
    sub_df.to_csv("output/{}.csv".format(name), index=False)

    print("score: {:.8f}, MAE: {}".format(score(oof, y_train), MAE(oof, y_train)))
    
    return clf

In [ ]:
# model_1
X_train = train_df.iloc[:, 1:-1].values
y_train = target.values
X_test = test_df.iloc[:, 1:].values

param = {'num_leaves': 30,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}

clf = lgb_model(param, X_train, y_train, X_test, 'model_1_test')

In [ ]:
feature_select = pd.DataFrame({'importance': clf.feature_importance()}, index= train_df.iloc[:, 1:-1].columns,).sort_values(by='importance', ascending=False)